In [1]:
import pandas as pd
import numpy as np
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
df = pd.read_csv('Crime_Data_from_2020_to_Present_20240111.csv')
df.head()

,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,...,Status,Status Desc,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON
0,10304468,01/08/2020 12:00:00 AM,01/08/2020 12:00:00 AM,2230,3,Southwest,377,2,624,BATTERY - SIMPLE ASSAULT,...,AO,Adult Other,624.0,NaN,NaN,NaN,1100 W 39TH PL,NaN,34.0141,-118.2978
1,190101086,01/02/2020 12:00:00 AM,01/01/2020 12:00:00 AM,330,1,Central,163,2,624,BATTERY - SIMPLE ASSAULT,...,IC,Invest Cont,624.0,NaN,NaN,NaN,700 S HILL ST,NaN,34.0459,-118.2545
2,200110444,04/14/2020 12:00:00 AM,02/13/2020 12:00:00 AM,1200,1,Central,155,2,845,SEX OFFENDER REGISTRANT OUT OF COMPLIANCE,...,AA,Adult Arrest,845.0,NaN,NaN,NaN,200 E 6TH ST,NaN,34.0448,-118.2474
3,191501505,01/01/2020 12:00:00 AM,01/01/2020 12:00:00 AM,1730,15,N Hollywood,1543,2,745,VANDALISM - MISDEAMEANOR ($399 OR UNDER),...,IC,Invest Cont,745.0,998.0,NaN,NaN,5400 CORTEEN PL,NaN,34.1685,-118.4019
4,191921269,01/01/2020 12:00:00 AM,01/01/2020 12:00:00 AM,415,19,Mission,1998,2,740,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",...,IC,Invest Cont,740.0,NaN,NaN,NaN,14400 TITUS ST,NaN,34.2198,-118.4468


In [3]:
df.shape

(764061, 28)

In [4]:
# rename the columns so they are more easily understood
df.columns = ['Div Record No', 'Date Reported', 'Date Occurred', 'Time Occurred', 'Area Code', 'Area Name', 'Sub Area Code', 'Part 1-2', 'Crime Code', 'Crime Desc',\
              'Mocodes', 'Vict Age', 'Vict Sex', 'Vict Descent', 'Premise Code', 'Premise Desc', 'Weapon Code', 'Weapon Desc', 'Status Code', 'Status Desc', \
                'Crime Code 1', 'Crime Code 2', 'Crime Code 3', 'Crime Code 4', 'Address', 'Cross St', 'Latitude', 'Longitude']

In [5]:
# count the number of duplicate rows
df.duplicated().sum()

171895

In [6]:
# drop any rows with duplicate values
df.drop_duplicates(inplace=True)
df.duplicated().sum()
df.shape

0

(592166, 28)

In [7]:
# check for data types
df.dtypes

Div Record No      int64
Date Reported     object
Date Occurred     object
Time Occurred      int64
Area Code          int64
Area Name         object
Sub Area Code      int64
Part 1-2           int64
Crime Code         int64
Crime Desc        object
Mocodes           object
Vict Age           int64
Vict Sex          object
Vict Descent      object
Premise Code     float64
Premise Desc      object
Weapon Code      float64
Weapon Desc       object
Status Code       object
Status Desc       object
Crime Code 1     float64
Crime Code 2     float64
Crime Code 3     float64
Crime Code 4     float64
Address           object
Cross St          object
Latitude         float64
Longitude        float64
dtype: object

In [8]:
# set up the dates column to be in the correct format
df[['Date Reported', 'Date Occurred']] = df[['Date Reported', 'Date Occurred']].apply(pd.to_datetime)

In [9]:
# Learned how to pad zeroes to numbers thanks to https://stackoverflow.com/questions/51905926/python-how-to-create-a-string-that-is-4-characters-long-from-a-number
# Note: if the time is recorded as 1, then it means 1 minute past midnight.
# We need to add a leading zero to make it 0001 so we can convert it into the time format 00:01 (24 hour time)
df['Time Occurred'] = df['Time Occurred'].apply(lambda x: "{:04d}".format(x))
df['Time Occurred'] = pd.to_datetime(df['Time Occurred'], format = '%H%M', errors='coerce').dt.time

In [10]:
df[['Area Code', 'Sub Area Code', 'Crime Code', 'Premise Desc', 'Weapon Desc']] = \
    df[['Area Code', 'Sub Area Code', 'Crime Code', 'Premise Desc', 'Weapon Desc']].astype(str)

In [13]:
# fill in missing values with -1, then convert the column values from float to int, then convert the -1 values 
df.fillna({'Premise Code':-1, 'Weapon Code':-1, 'Crime Code 1':-1, 'Crime Code 2':-1,'Crime Code 3':-1,'Crime Code 4':-1}, inplace=True)

df[['Premise Code', 'Weapon Code', 'Crime Code 1', 'Crime Code 2','Crime Code 3','Crime Code 4']] = \
    df[['Premise Code', 'Weapon Code', 'Crime Code 1', 'Crime Code 2','Crime Code 3','Crime Code 4']].astype(int).astype(str)

df[['Premise Code', 'Weapon Code', 'Crime Code 1', 'Crime Code 2','Crime Code 3','Crime Code 4']] = \
    df[['Premise Code', 'Weapon Code', 'Crime Code 1', 'Crime Code 2','Crime Code 3','Crime Code 4']].replace('-1', '')

In [14]:
# clean up the trailing '*' in the Premise Desc column
df['Premise Desc'] = df['Premise Desc'].apply(lambda x : x.replace('*', '').strip())

# fill in the missing values in the Premise Desc column with 'UNKNOWN'
df['Premise Desc'] = df['Premise Desc'].replace('nan', 'UNKNOWN')
df['Premise Desc'] = df['Premise Desc'].replace(np.nan, 'UNKNOWN')

# fill in the missing values in the Weapon Desc column with 'NONE'
df['Weapon Desc'] = df['Weapon Desc'].replace('nan', 'NONE')
df['Weapon Desc'] = df['Weapon Desc'].replace(np.nan, 'NONE')


In [15]:
df.dtypes

Div Record No             int64
Date Reported    datetime64[ns]
Date Occurred    datetime64[ns]
Time Occurred            object
Area Code                object
Area Name                object
Sub Area Code            object
Part 1-2                  int64
Crime Code               object
Crime Desc               object
Mocodes                  object
Vict Age                  int64
Vict Sex                 object
Vict Descent             object
Premise Code             object
Premise Desc             object
Weapon Code              object
Weapon Desc              object
Status Code              object
Status Desc              object
Crime Code 1             object
Crime Code 2             object
Crime Code 3             object
Crime Code 4             object
Address                  object
Cross St                 object
Latitude                float64
Longitude               float64
dtype: object

In [16]:
df.isna().any()

Div Record No    False
Date Reported    False
Date Occurred    False
Time Occurred    False
Area Code        False
Area Name        False
Sub Area Code    False
Part 1-2         False
Crime Code       False
Crime Desc       False
Mocodes           True
Vict Age         False
Vict Sex          True
Vict Descent      True
Premise Code     False
Premise Desc     False
Weapon Code      False
Weapon Desc      False
Status Code       True
Status Desc      False
Crime Code 1     False
Crime Code 2     False
Crime Code 3     False
Crime Code 4     False
Address          False
Cross St          True
Latitude         False
Longitude        False
dtype: bool

In [17]:
# For example, check the rows where the column 'Weapon Desc' is NaN
a = df[df['Weapon Desc'].isna()]
a.loc[:, 'Weapon Desc']

Series([], Name: Weapon Desc, dtype: object)

In [18]:
# fill null values with an empty string (for consistency with the original dataset)
df.fillna('', inplace=True)

In [19]:
df.isna().any()

Div Record No    False
Date Reported    False
Date Occurred    False
Time Occurred    False
Area Code        False
Area Name        False
Sub Area Code    False
Part 1-2         False
Crime Code       False
Crime Desc       False
Mocodes          False
Vict Age         False
Vict Sex         False
Vict Descent     False
Premise Code     False
Premise Desc     False
Weapon Code      False
Weapon Desc      False
Status Code      False
Status Desc      False
Crime Code 1     False
Crime Code 2     False
Crime Code 3     False
Crime Code 4     False
Address          False
Cross St         False
Latitude         False
Longitude        False
dtype: bool

In [20]:
# Replace the blank values with 'X', which stands for unknown
df['Vict Descent'].replace(['', '-'], 'X', inplace=True)
df['Vict Sex'].replace(['', '-'], 'X', inplace=True)

In [21]:
# remove negative values in the "Vict Age" column
age_idx = df[df['Vict Age'] < 0].index
df.drop(age_idx, inplace=True)

In [22]:
# create a new column that reports how long it took for the crime to be reported (in days)
df['Report Delay'] = df['Date Reported'] - df['Date Occurred']
df['Report Delay'] = df['Report Delay'].dt.days

In [23]:
# bin times of day a crime occurred based on the hour it occurred
b = [0, 4, 12, 17, 21, 24]
l = ['Late Night (0-4)', 'Morning (5-12)', 'Afternoon (13-17)', 'Evening (18-21)', 'Night (22-24)']
df['Time of Day'] = pd.cut(df['Time Occurred'].apply(lambda x: x.hour), bins=b, labels=l, include_lowest=True)

In [24]:
# bin the age range of the victim
b = [0, 18, 24, 35, 55, 65, 100]
l = ['Under 18', '18-24', '25-35', '36-55', '55-65', 'Over 65']
df['Vict Age Range'] = pd.cut(df['Vict Age'], bins=b, labels=l, include_lowest=True)

In [25]:
# move the last three columns and shift them to the right order
# reference to get index of a particular column: https://stackoverflow.com/questions/13021654/get-column-index-from-column-name-in-python-pandas
report_delay = df.pop('Report Delay')
df.insert(df.columns.get_loc("Date Occurred") + 1, 'Report Delay', report_delay)

time_of_day = df.pop('Time of Day')
df.insert(df.columns.get_loc("Time Occurred") + 1, 'Time of Day', time_of_day)

vict_age_range = df.pop('Vict Age Range')
df.insert(df.columns.get_loc("Vict Age") + 1, 'Vict Age Range', vict_age_range)

In [26]:
# combine the "Address" and "Cross St" columns together into a new location column
df['New Location'] = df.apply(lambda x: x['Address'] if x['Cross St'] == '' else x['Address'] + ' & ' + x['Cross St'], axis=1)
# drop the original columns
df.drop(['Address', 'Cross St'], axis=1, inplace=True)
# rename the new column as "Location"
df.rename(columns={'New Location': 'Location'}, inplace=True)
# remove trailing space in between the words
df['Location'] = df['Location'].apply(lambda x: " ".join(x.split()))
df.head()

,Div Record No,Date Reported,Date Occurred,Report Delay,Time Occurred,Time of Day,Area Code,Area Name,Sub Area Code,Part 1-2,...,Weapon Desc,Status Code,Status Desc,Crime Code 1,Crime Code 2,Crime Code 3,Crime Code 4,Latitude,Longitude,Location
0,10304468,2020-01-08,2020-01-08,0,22:30:00,Night (22-24),3,Southwest,377,2,...,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",AO,Adult Other,624,,,,34.0141,-118.2978,1100 W 39TH PL
1,190101086,2020-01-02,2020-01-01,1,03:30:00,Late Night (0-4),1,Central,163,2,...,UNKNOWN WEAPON/OTHER WEAPON,IC,Invest Cont,624,,,,34.0459,-118.2545,700 S HILL ST
2,200110444,2020-04-14,2020-02-13,61,12:00:00,Morning (5-12),1,Central,155,2,...,NONE,AA,Adult Arrest,845,,,,34.0448,-118.2474,200 E 6TH ST
3,191501505,2020-01-01,2020-01-01,0,17:30:00,Afternoon (13-17),15,N Hollywood,1543,2,...,NONE,IC,Invest Cont,745,998,,,34.1685,-118.4019,5400 CORTEEN PL
4,191921269,2020-01-01,2020-01-01,0,04:15:00,Late Night (0-4),19,Mission,1998,2,...,NONE,IC,Invest Cont,740,,,,34.2198,-118.4468,14400 TITUS ST


In [ ]:
df.dtypes

Div Record No              int64
Date Reported     datetime64[ns]
Date Occurred     datetime64[ns]
Report Delay               int64
Time Occurred             object
Time of Day             category
Area Code                 object
Area Name                 object
Sub Area Code             object
Part 1-2                   int64
Crime Code                object
Crime Desc                object
Mocodes                   object
Vict Age                   int64
Vict Age Range          category
Vict Sex                  object
Vict Descent              object
Premise Code              object
Premise Desc              object
Weapon Code               object
Weapon Desc               object
Status Code               object
Status Desc               object
Crime Code 1              object
Crime Code 2              object
Crime Code 3              object
Crime Code 4              object
Latitude                 float64
Longitude                float64
Location                  object
dtype: obj

In [27]:
df = df.reset_index(drop=True)

In [28]:
crimes = df[['Crime Code', 'Crime Desc']]
crimes.drop_duplicates(inplace=True)

crimes['Crime Code'] = crimes['Crime Code'].astype(int)
crimes = crimes.sort_values(by=['Crime Code']).reset_index(drop=True)

crimes['Crime Code'] = crimes['Crime Code'].astype(str)
crimes

/var/folders/9x/x8dkd_c96r10bmctrjkv3zyc0000gn/T/ipykernel_93252/2764548920.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crimes.drop_duplicates(inplace=True)
/var/folders/9x/x8dkd_c96r10bmctrjkv3zyc0000gn/T/ipykernel_93252/2764548920.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crimes['Crime Code'] = crimes['Crime Code'].astype(int)


,Crime Code,Crime Desc
0,110,CRIMINAL HOMICIDE
1,113,"MANSLAUGHTER, NEGLIGENT"
2,121,"RAPE, FORCIBLE"
3,122,"RAPE, ATTEMPTED"
4,210,ROBBERY
...,...,...
133,949,ILLEGAL DUMPING
134,950,"DEFRAUDING INNKEEPER/THEFT OF SERVICES, OVER $..."
135,951,"DEFRAUDING INNKEEPER/THEFT OF SERVICES, $950 &..."
136,954,CONTRIBUTING


In [29]:
areas = df[['Area Code', 'Area Name']]
areas.drop_duplicates(inplace=True)

areas['Area Code'] = areas['Area Code'].astype(int)
areas = areas.sort_values(by=['Area Code']).reset_index(drop=True)

areas['Area Code'] = areas['Area Code'].astype(str)
areas

/var/folders/9x/x8dkd_c96r10bmctrjkv3zyc0000gn/T/ipykernel_93252/4116095878.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  areas.drop_duplicates(inplace=True)
/var/folders/9x/x8dkd_c96r10bmctrjkv3zyc0000gn/T/ipykernel_93252/4116095878.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  areas['Area Code'] = areas['Area Code'].astype(int)


,Area Code,Area Name
0,1,Central
1,2,Rampart
2,3,Southwest
3,4,Hollenbeck
4,5,Harbor
5,6,Hollywood
6,7,Wilshire
7,8,West LA
8,9,Van Nuys
9,10,West Valley


In [30]:
ethnicity_map = {
    'A': 'Other Asian',
    'B': 'Black',
    'C': 'Chinese',
    'D': 'Cambodian',
    'F': 'Filipino',
    'G': 'Guamanian',
    'H': 'Hispanic/Latin/Mexican',
    'I': 'American Indian/Alaskan Native',
    'J': 'Japanese',
    'K': 'Korean',
    'L': 'Laotian',
    'O': 'Other',
    'P': 'Pacific Islander',
    'S': 'Samoan',
    'U': 'Hawaiian',
    'V': 'Vietnamese',
    'W': 'White',
    'X': 'Unknown',
    'Z': 'Asian Indian'
}
vict_descent = df['Vict Descent'].drop_duplicates()
vict_descent = vict_descent.to_frame().sort_values(by=['Vict Descent']).reset_index(drop=True)
vict_descent['Ethnicity'] = vict_descent['Vict Descent'].apply(lambda x: ethnicity_map[x])
vict_descent

,Vict Descent,Ethnicity
0,A,Other Asian
1,B,Black
2,C,Chinese
3,D,Cambodian
4,F,Filipino
5,G,Guamanian
6,H,Hispanic/Latin/Mexican
7,I,American Indian/Alaskan Native
8,J,Japanese
9,K,Korean


In [31]:
premise = df[['Premise Code', 'Premise Desc']]
premise.drop_duplicates(inplace=True)

code_idx = premise[premise['Premise Code'] == ''].index
premise.drop(code_idx, inplace=True)

# we could remove the rows where the premise description is blank, but then we wouldn't be able to join on codes where the descriptions are blank
# desc_idx = premise[premise['Premise Desc'] == ''].index
# premise.drop(desc_idx, inplace=True)

premise['Premise Code'] = premise['Premise Code'].astype(int)
premise = premise.sort_values(by=['Premise Code']).reset_index(drop=True)

premise['Premise Code'] = premise['Premise Code'].astype(str)
premise

/var/folders/9x/x8dkd_c96r10bmctrjkv3zyc0000gn/T/ipykernel_93252/535025749.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  premise.drop_duplicates(inplace=True)
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/core/frame.py:4305: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
/var/folders/9x/x8dkd_c96r10bmctrjkv3zyc0000gn/T/ipykernel_93252/535025749.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydat

,Premise Code,Premise Desc
0,101,STREET
1,102,SIDEWALK
2,103,ALLEY
3,104,DRIVEWAY
4,105,PEDESTRIAN OVERCROSSING
...,...,...
309,971,MTA - GOLD LINE - HIGHLAND PARK
310,972,UNKNOWN
311,973,UNKNOWN
312,974,UNKNOWN


In [32]:
weapons = df[['Weapon Code', 'Weapon Desc']]
weapons.drop_duplicates(inplace=True)
weapons = weapons.sort_values(by=['Weapon Code']).reset_index(drop=True)
weapons.reset_index(drop=True, inplace=True)
weapons

/var/folders/9x/x8dkd_c96r10bmctrjkv3zyc0000gn/T/ipykernel_93252/3340642964.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weapons.drop_duplicates(inplace=True)


,Weapon Code,Weapon Desc
0,,NONE
1,101,REVOLVER
2,102,HAND GUN
3,103,RIFLE
4,104,SHOTGUN
...,...,...
72,512,MACE/PEPPER SPRAY
73,513,STUN GUN
74,514,TIRE IRON
75,515,PHYSICAL PRESENCE


In [33]:
case_status = df[['Status Code', 'Status Desc']]
case_status.drop_duplicates(inplace=True)
case_status = case_status.sort_values(by=['Status Code']).reset_index(drop=True)
case_status

/var/folders/9x/x8dkd_c96r10bmctrjkv3zyc0000gn/T/ipykernel_93252/1618493048.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  case_status.drop_duplicates(inplace=True)


,Status Code,Status Desc
0,,UNK
1,AA,Adult Arrest
2,AO,Adult Other
3,CC,UNK
4,IC,Invest Cont
5,JA,Juv Arrest
6,JO,Juv Other


In [34]:
# write the cleaned data to an csv file, which we can use for our Tableau visualization
df.to_csv('cleaned_crime_data.csv', index=False)
vict_descent.to_csv('ethnicity.csv', index=False)